In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np 
from scipy import signal 
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
from keras import regularizers

In [ ]:
x_train = np.load('../BraiNeoCare/Datasets/GAT/traindata.npy',mmap_mode='r')
y_train = np.load('../BraiNeoCare/Datasets/GAT/trainlabels.npy',mmap_mode='r')
x_test  = np.load('../BraiNeoCare/Datasets/GAT/testdata.npy',mmap_mode='r')
y_test  = np.load('../BraiNeoCare/Datasets/GAT/testlabels.npy',mmap_mode='r')

In [ ]:
mean=x_train.mean()
std=x_train.std()

x_train=(x_train-mean)/std
x_test=(x_test-mean)/std

train_Aug = np.flip(x_train,axis=2)
for r in range(train_Aug.shape[0]):
    random_noise=np.random.normal(0,0.001,(12,384))
    train_Aug[r]+=random_noise
x_train=np.expand_dims(x_train,axis=-1)
x_test=np.expand_dims(x_test,axis=-1)
# x1=np.expand_dims(x1,axis=-1)

np.random.seed(25)
train_indices = np.arange(x_train.shape[0])
np.random.shuffle(train_indices)
x_train = x_train[train_indices]
y_train = y_train[train_indices]


In [ ]:
## for 18 channels 
adj=tf.constant(
    [  [1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.]],
    dtype=tf.float32)

In [ ]:
channel_names=["Fp1-T3","T3-O1","Fp1-C3","C3-O1","Fp2-C4","C4-O2","Fp2-T4","T4-O2","T3-C3","C3-Cz","Cz-C4","C4-T4"]
indices =[[r,i] for r,c1 in enumerate(channel_names) for i,c2 in enumerate(channel_names) if (c1.split("-")[0]==c2.split("-")[1] or c1.split("-")[1]==c2.split("-")[1] 
          or c1.split("-")[0]==c2.split("-")[0] or c1.split("-")[1]==c2.split("-")[0])]
adj=np.zeros((12,12))
for i in indices:
    adj[i[0]][i[1]]=1
adj=tf.constant(adj,dtype=tf.float32)

class GATLayer(layers.Layer):

    def __init__(self,output_dim):
        super(GATLayer, self).__init__()
        self.output_dim = output_dim
        self.Leakyrelu = layers.LeakyReLU(alpha=0.2)
    
    def build(self, input_shape):
        self.W = self.add_weight(name='W',shape=(input_shape[-1], self.output_dim), initializer='random_normal',trainable=True)
        self.a = self.add_weight(name='a',shape=(2*self.output_dim, 1), initializer='random_normal',trainable=True)
    
    def call(self,input,adj):
        H= tf.matmul(input, self.W)
        h1=tf.tile(tf.expand_dims(H, axis=1), [1,12,1,1])
        h2=tf.tile(tf.expand_dims(H, axis=2), [1,1,12,1])
        result =tf.concat([h1 , h2], axis=-1)
        e=self.Leakyrelu(tf.squeeze(tf.matmul(result, self.a),axis=-1))
        zero_mat= -1e20*tf.ones_like(e)
        msked_e=tf.where(adj==1.0,e,zero_mat)
        alpha=tf.nn.softmax(msked_e,axis=-1)
        HPrime=tf.matmul(alpha,H)
        return tf.nn.elu(HPrime)

def create_model():
    Input= keras.Input(shape=(12,384,1))
    regularizer_dense=regularizers.l2(0.0001)

    x= layers.Conv2D(32,(1,5),activation='relu',padding='same')(Input)
    y= layers.Conv2D(32,(1,7),activation='relu',padding='same')(Input)
    x= layers.add([x,y])
    x= layers.AveragePooling2D((1,2))(x)
    x= layers.BatchNormalization()(x)
    x= layers.SpatialDropout2D(0.2)(x)

    x= layers.Conv2D(64,(1,5),activation='relu',padding='same')(x)
    y= layers.Conv2D(64,(1,7),activation='relu',padding='same')(x)
    x= layers.add([x,y])
    x= layers.AveragePooling2D((1,2))(x)
    x= layers.BatchNormalization()(x)
    x= layers.SpatialDropout2D(0.2)(x)

    x= layers.Conv2D(8,(1,5),activation='relu',padding='same')(x)
    y= layers.SpatialDropout2D(0.2)(x)
    y= layers.Conv2D(8,(1,7),activation='relu',padding='same')(y)
    x= layers.add([x,y])
    x= layers.AveragePooling2D((1,2))(x)
    x= layers.BatchNormalization()(x)
    x= layers.SpatialDropout2D(0.2)(x)

    x= layers.Conv2D(1,(1,5),activation='relu',padding='same')(x)
    y= layers.SpatialDropout2D(0.2)(x)
    y= layers.Conv2D(1,(1,7),activation='relu',padding='same')(y)
    x= layers.add([x,y])
    x= layers.AveragePooling2D((1,2))(x)
    x= layers.Reshape((12,24))(x)

    x= GATLayer(37)(x,adj)
    x= GATLayer(32)(x,adj)
    x= GATLayer(16)(x,adj)

    x= layers.GlobalAveragePooling1D()(x)
    x= layers.Dropout(0.2)(x)
    x= layers.Dense(32,activation='relu',kernel_regularizer=regularizer_dense)(x)
    x= layers.Dropout(0.2)(x)
    x= layers.Dense(16,activation='relu',kernel_regularizer=regularizer_dense)(x)
    x= layers.Dropout(0.2)(x)
    x= layers.Dense(1,activation='sigmoid')(x)

    model = keras.Model(inputs=Input, outputs=x)

    optimizer=keras.optimizers.Adam(learning_rate=0.002)
    loss=keras.losses.BinaryFocalCrossentropy(from_logits=False,gamma=2,alpha=0.4,apply_class_balancing=True)
    # loss=keras.losses.BinaryCrossentropy(from_logits=False,label_smoothing=0.001)  
    kappa=tfa.metrics.CohenKappa(num_classes=2)
    fp=keras.metrics.FalsePositives()
    tn=keras.metrics.TrueNegatives()
    precision = keras.metrics.Precision()
    recall = keras.metrics.Recall()
    AUROC = keras.metrics.AUC(curve='ROC', name = 'AUROC')
    AUPRC = keras.metrics.AUC(curve='PR', name = 'AUPRC')
    model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy', AUROC, AUPRC,fp,tn, precision, recall,kappa])   
    return model
model=create_model()


In [ ]:
# checkpoint_path = "GAT_correct_remArt_3/cp_{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path) 
# cp_callback=keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=False,verbose=0,save_best_only=True,monitor='val_accuracy')  
history=model.fit(x_train,y_train,epochs=200,batch_size=512,verbose=1,validation_data=(x_test,y_test))

In [ ]:
with open("history_GAT_correct_8.jason", 'w') as f:
    pd.DataFrame(history.history).to_json(f)

In [ ]:
with open('history_cv_0.jason','r') as f:
    history_1 = pd.read_json(f)
with open('history_cv_1.jason','r') as f:
    history_2 = pd.read_json(f)
with open('history_cv_2.jason','r') as f:
    history_3 = pd.read_json(f)
with open('history_cv_3.jason','r') as f:
    history_4 = pd.read_json(f)
with open('history_cv_4.jason','r') as f:
    history_5 = pd.read_json(f)

np.percentile([history_1['val_accuracy'].max(),history_2['val_accuracy'].max(),history_3['val_accuracy'].max(),history_4['val_accuracy'].max(),history_5['val_accuracy'].max()],[75,25])

In [ ]:
metrics=['accuracy','val_accuracy','loss','val_loss','val_AUROC','val_AUPRC','val_precision','val_recall','recall','precision','AUROC','AUPRC']
epochs = range(1, 101)
fig,ax=plt.subplots(4,3,figsize=(20,20))
for r in range(12):
    ax[r//3][r%3].plot(epochs,history_2[metrics[r]],label=metrics[r],color='r')
    ax[r//3][r%3].set_title(metrics[r])
    ax[r//3][r%3].set_xlabel('Epochs')
    ax[r//3][r%3].grid()
fig.tight_layout()
plt.show()